In [8]:
import pandas as pd
import numpy as np

import torch 
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn

In [2]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [3]:
x_test = test_df.values/255
y = train_df.label.values
x = train_df.iloc[:,1:].values/255
train_x, val_x, train_y, val_y = train_test_split(x, y, test_size = 0.2) 

In [4]:
trainTorch_x = torch.from_numpy(train_x).type(torch.FloatTensor)
trainTorch_y = torch.from_numpy(train_y).type(torch.LongTensor)


valTorch_x = torch.from_numpy(val_x).type(torch.FloatTensor)
valTorch_y = torch.from_numpy(val_y).type(torch.LongTensor) 

testTorch_x = torch.from_numpy(np.array(x_test)).type(torch.FloatTensor)
# testTorch_x = testTorch_x.to(device)

In [5]:
train = torch.utils.data.TensorDataset(trainTorch_x, trainTorch_y)
val = torch.utils.data.TensorDataset(valTorch_x, valTorch_y)

In [6]:
train_loader = DataLoader(train, batch_size = 100, shuffle = False)
val_loader = DataLoader(val, batch_size = 100, shuffle = False)

In [18]:
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA,self).__init__()
#         branch one
        self.branch1x1 = nn.Conv2d(in_channels=in_channels,out_channels=16,kernel_size=1)

#         branch two
        self.branch5x5_1 = nn.Conv2d(in_channels=in_channels,out_channels=16,kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(in_channels=16,out_channels=24,kernel_size=5,padding=2)
        
#         branch three
        self.branch3x3_1 = nn.Conv2d(in_channels=in_channels,out_channels=16,kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16,24,kernel_size=3,padding=1)
        self.branch3x3_3 = nn.Conv2d(24,24,kernel_size=3,padding=1)
        
#         branch four
        self.branch_pool = nn.Conv2d(in_channels,24,kernel_size=1)
    
    def forward(self,x):
        
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        
        branch_pool = F.avg_pool2d(x,kernel_size=3,stride=1,padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1,branch5x5,branch3x3,branch_pool]
        return torch.cat(outputs,dim = 1)

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = nn.Conv2d(88,20,kernel_size=5)
        
        self.incep1 = InceptionA(in_channels=10)
        self.incep2 = InceptionA(in_channels=20)
        
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(1408,512)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128,10)
        
    def forward(self,x):
        in_size = x.size(0)
        x = x.view(in_size,1,28,28)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size,-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [26]:
model = Net()
# model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum=0.5)

In [27]:
# 将单轮循环封装为一个函数

def train_func(epoch):
    running_loss = 0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
#         inputs,target = inputs.to(device),target.to(device)
        optimizer.zero_grad()
        
        
#         forward and backward and update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d,%5d] loss: %.3f' % (epoch + 1,batch_idx + 1,running_loss / 300))
            running_loss =0.0
            

In [28]:
def test_func():
    correct = 0
    total = 0
    with torch.no_grad():
#         执行之后在下面代码就不会执行梯度
        for data in val_loader:
            images,labels = data
#             images,labels = images.to(device),labels.to(device)
#             拿数据
            outputs = model(images)
#             做预测，拿到的结果是一个矩阵，每一行都是一个独热向量
            _, predicted = torch.max(outputs.data,dim = 1)
#           返回 最大值 和 每一行的最大值下标
#           指定沿着维度1（往下 行是第0个维度，向右 列是第一个维度）
            total += labels.size(0)
#             label是一个N 1元组 size 取 0 就是？

            correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %d %%' % (100 * correct / total))

In [29]:
def predict_func():
    with torch.no_grad():
        outputs = model(testTorch_x)
        index, predicted = torch.max(outputs.data,dim = 1)
    ans2 = predicted
    submission_file2 = pd.read_csv('./sample_submission.csv')
    submission_file2.Label = ans2
    submission_file2.to_csv('submission_cnn_googlenet.csv', index=False)
    print("saved")

In [32]:
if __name__ == '__main__':
    for epoch in range(1):
        train_func(epoch)
        test_func()
    predict_func()

[1,  300] loss: 0.000
Accuracy on test set: 98 %
saved
